In [33]:
import re
import numpy as np
import pandas as pd
import xlwt
import os
import ete3
ncbi = ete3.NCBITaxa()
import requests
#ncbi.update_taxonomy_database()

In [34]:
BASE = 'http://www.uniprot.org'
KB_ENDPOINT = '/uniprot/'
TOOL_ENDPOINT = '/uploadlists/'

Choose species

In [35]:
species ="Francisella tularensis"

Read in exported data files

In [36]:
HPIDB = pd.read_table("/Users/thesis/Desktop/Google drive/thesis-lorenz/Work in progress.../Databases/HPIDB2.0/Mitabs/" + species + "/" + species + ".txt", names=["Protein_A","Protein_B","Alt_A","Alt_B","Alias_A","Alias_B","D_method","PubmedID","Taxid_A","Taxid_B","Int_type","Source_DB","Conf_score"], usecols=[0,1,2,3,4,5,6,8,9,10,11,12,14], header=0)
IntAct = pd.read_table("/Users/thesis/Desktop/Google drive/thesis-lorenz/Work in progress.../Databases/IntAct/" + species, names=["Protein_A","Protein_B","Alt_A","Alt_B","Alias_A","Alias_B","D_method","PubmedID","Taxid_A","Taxid_B","Int_type","Source_DB","Conf_score"], usecols=[0,1,2,3,4,5,6,8,9,10,11,12,14], header=0)
PHISTO = pd.read_excel("/Users/thesis/Desktop/Google drive/thesis-lorenz/Work in progress.../Databases/PHISTO/" + species + ".xls", names= ["Taxid_A", "Protein_A", "Protein_B", "D_method","PubmedID"], usecols=[1,2,4,6,7])
print(HPIDB.Source_DB.unique())
print (len(HPIDB), len(IntAct),len(PHISTO))

['psi-mi:MI:0469(IntAct)']
1371 1358 1331


IntAct and HPIDB sets are basically the same apart from their taxid columns, thus we'll clean up the taxid column of IntAct, merge the two and then clean up the rest

It should also be noted that the taxid cleaning up has one step that needs to 'supervised' because sometimes the procedures creates NaN values which need to be checked manually.

In [37]:
r = IntAct.Taxid_A
r = r.str.lower()
r = r.str.extract(r'(?:taxid:)([0-9]*)',expand = False)
IntAct.Taxid_A = r

r = IntAct.Taxid_B
r = r.str.lower()
r = r.str.extract(r'(?:taxid:)([0-9]*)',expand = False)
IntAct.Taxid_B = r

In [38]:
df=pd.concat([HPIDB,IntAct])
df=df.reset_index(drop=True)

In [39]:
len(df)

2729

In [40]:
taxids= list(pd.unique(list(df.Taxid_A.unique())+list(pd.unique(list(df.Taxid_B.unique())))))
parent= ncbi.get_name_translator([species])
taxid_list=[]
for taxid in taxids:
    lineage = ncbi.get_lineage(taxid)
    
    if parent.get(species)[0] in lineage:
        taxid_list.append(taxid)
taxid_list.append('9606')

In [41]:
print(ncbi.get_taxid_translator(taxid_list))
sel1 = df.Taxid_A.apply(lambda x: str(x) in taxid_list)
sel2 = df.Taxid_B.apply(lambda x: str(x) in taxid_list)
df = df.loc[sel1 & sel2,:]

{9606: 'Homo sapiens', 119856: 'Francisella tularensis subsp. tularensis', 177416: 'Francisella tularensis subsp. tularensis SCHU S4'}


In [42]:
len(df)

2729

In [43]:
for name in ["D_method","Int_type","Source_DB"]:
    r = df[name]
    r = r.str.extract((r'(?:psi-mi:"?MI:[0-9]*"?[(])([^)]*)(?:[)])') ,expand = False)
    df[name] = r


for name in ["Protein_A","Protein_B"]:
    r = df[name]
    r = r.str.extract(r'(?:uniprotkb:)([a-zA-Z0-9]*)', expand=False)
    df[name] = r
    

r = df.PubmedID
r = r.str.extract(r'(?:[^p]*)(?:pubmed:)([0-9]*)(?:.*)',expand=False)
df.PubmedID = r

r = df.Conf_score
r = r.str.extract(r'(?:intact-miscore:)([0-9.]*)', expand=False)
df.Conf_score = r

Some entries don't have uniprot ID
We'll search the alt ID column for a uniprot ID
However, in the particular case of B anthracis, there are no Alt Ids available => entries 714-761 in intact file for protein A
Protein B is spread across the dataset. 

Upon further inspection most of these entries trace back do deleted entries in the uniprotKB database.
    Some of these due to deletion. We don't know (yet) if this deletion is due to being wrong or upon request of original submitters
        Others due to redundancy. Trace back to the non redundant entry might be done by downloading the sequence and finding it in the uniprot database? example entry index 56 of intact file
        EBI-2810906 => a0a1v4b700_bacan => 
The other entries aren't even traceable

In [44]:
def Find_alt_ID(ID,organism):
    payload = {'query': 'name:' + ID + 'AND organism:' + organism + 'AND reviewed:yes',
           'format': 'list'}

    result2 = requests.get(BASE + KB_ENDPOINT, params=payload)

    if result2.ok:
        print(result2.text)
    else:
        print('Something went wrong ', result.status_code)

In [45]:
sel1 = df.Protein_A.isnull()
subset1 = df.loc[sel1,"Alias_A"]
sel2 = df.Protein_B.isnull()
subset2 = df.loc[sel2,"Alias_B"]

pandas.core.series.Series

In [15]:
sel1 = df.Protein_A.isnull()
sel2 = df.Protein_B.isnull()
df = df.loc[~sel1 & ~sel2,:]
df = df.reset_index(drop=True)

In [16]:
len(df)

2637

In [17]:
df = df.drop(["Alt_A","Alt_B"],axis=1)

Sort columns so pathogen is 'A' and human is 'B'

In [18]:
df.Taxid_A= df.Taxid_A.apply(lambda x: str(x))
df.Taxid_B = df.Taxid_B.apply(lambda x: str(x))
for i in range(len(df)):
    if df.loc[i,"Taxid_B"] != '9606':
        A = df.loc[i,"Taxid_A"]
        B = df.loc[i,"Taxid_B"]
        C = df.loc[i,"Protein_A"]
        D = df.loc[i,"Protein_B"]
      

        df.loc[i,"Taxid_A"] = B
        df.loc[i,"Taxid_B"] = A
        df.loc[i,"Protein_A"]= D
        df.loc[i,"Protein_B"]= C

Filter out non-HPIs

In [19]:
sel = df.Taxid_B == '9606'

In [20]:
df = df.loc[sel,:]
print(len(df))

2637


Make identifier df

In [21]:
ID_all =[]
for i in range (0,len(df)):
    ID_list=[]
    ID_list.append (df.iloc[i,0])
    ID_list.append (df.iloc[i,1])
    ID_list.append (df.iloc[i,3])
    ID_list.append (df.iloc[i,2])
    ID_list.sort()
    ID_all.append(
        ID_list[0]
        +ID_list[1] 
        + str(ID_list[2])
        +ID_list[3]
    )
df=df.assign(ID=ID_all)

In [22]:
print(len(df.ID.unique()))
df.PubmedID.unique()

2275


array(['20711500'], dtype=object)

Make identifier PHISTO and make Taxid_B column

In [23]:
ID_all =[]
for i in range (0,len(PHISTO)):
    ID_list=[]
    ID_list.append (PHISTO.iloc[i,1])
    ID_list.append (PHISTO.iloc[i,2])
    ID_list.append (PHISTO.iloc[i,3])
    ID_list.append (str(PHISTO.iloc[i,4]))
    ID_list.sort()
    ID_all.append(
        ID_list[0]
        +ID_list[1] 
        + ID_list[2]
        + ID_list[3]
    )
PHISTO=PHISTO.assign(ID=ID_all)

PHISTO["Taxid_B"]="9606"
PHISTO["Source_DB"]="PHISTO"

In [24]:
print(len(PHISTO.ID.unique()))
print(len(PHISTO))

1331
1331


merge df and PHISTO

In [25]:
df = pd.concat([df,PHISTO])
len(df)

3968

Remove redundancy

In [26]:
df = df.sort_values(by = "Conf_score", ascending = False)
df = df.drop_duplicates("ID")
print(len(df))

3606


Count how many times an interaction has been found with different methods, different types of interactions, different publications,...

In [27]:
count=df.groupby(["Source_DB"]).count()
count

,Alias_A,Alias_B,Conf_score,D_method,ID,Int_type,Protein_A,Protein_B,PubmedID,Taxid_A,Taxid_B
Source_DB,,,,,,,,,,,
IntAct,2275,2275,2275,2275,2275,2275,2275,2275,2275,2275,2275
PHISTO,0,0,0,1331,1331,0,1331,1331,1331,1331,1331


In [28]:
df.groupby("PubmedID").size()

PubmedID
20711500    1331
20711500    2275
dtype: int64

In [29]:
df = df.reset_index()
df = df[["Taxid_A","Taxid_B","Protein_A","Protein_B","Int_type","D_method","PubmedID","Source_DB","Conf_score","ID"]]

In [30]:
df.to_csv("/Users/thesis/Desktop/Python scripts/Test outputs/" + species, sep ="\t")

In [31]:
df

,Taxid_A,Taxid_B,Protein_A,Protein_B,Int_type,D_method,PubmedID,Source_DB,Conf_score,ID
0,119856,9606,Q8G8T7,P13598,physical association,two hybrid pooling approach,20711500,IntAct,0.37,P13598Q8G8T7psi-mi:icam2_human(display_long)|u...
1,177416,9606,Q5NID9,Q15650,physical association,two hybrid pooling approach,20711500,IntAct,0.37,Q15650Q5NID9psi-mi:eftu_fratt(display_long)|un...
2,177416,9606,Q5NF74,Q96A61,physical association,two hybrid pooling approach,20711500,IntAct,0.37,Q5NF74Q96A61psi-mi:q5nf74_fratt(display_long)|...
3,177416,9606,Q5NGC7,Q92547,physical association,two hybrid pooling approach,20711500,IntAct,0.37,Q5NGC7Q92547psi-mi:q5ngc7_fratt(display_long)|...
4,177416,9606,Q5NIP6,O14773,physical association,two hybrid pooling approach,20711500,IntAct,0.37,O14773Q5NIP6psi-mi:gata_fratt(display_long)|un...
5,177416,9606,Q5NEV2,O94842,physical association,two hybrid pooling approach,20711500,IntAct,0.37,O94842Q5NEV2psi-mi:q5nev2_fratt(display_long)|...
6,177416,9606,Q5NES5,Q8IYM9,physical association,two hybrid pooling approach,20711500,IntAct,0.37,Q5NES5Q8IYM9psi-mi:q5nes5_fratt(display_long)|...
7,177416,9606,Q5NEX4,Q8NDV7,physical association,two hybrid pooling approach,20711500,IntAct,0.37,Q5NEX4Q8NDV7psi-mi:q5nex4_fratt(display_long)|...
8,177416,9606,Q5NIP6,Q8NDV7,physical association,two hybrid pooling approach,20711500,IntAct,0.37,Q5NIP6Q8NDV7psi-mi:gata_fratt(display_long)|un...
9,177416,9606,Q5NHD1,O94826,physical association,two hybrid pooling approach,20711500,IntAct,0.37,O94826Q5NHD1psi-mi:q5nhd1_fratt(display_long)|...
